In [ ]:
import numpy as np
from scipy.sparse import spdiags, csr_matrix, find, issparse, coo_matrix
from scipy.sparse.linalg import spilu
from scipy.io import savemat
from sklearn.metrics import pairwise_distances
from google.colab import drive

Numpy

In [ ]:
def supernode(A, c):
    edges = np.zeros(len(c), dtype=int)
    for i, current_c in enumerate(c):
        F = A.copy()
        n = F.shape[0]
        m = len(current_c)
        E = np.zeros((m, n), dtype=int)
        prev = 0
        for j in range(m):
            later = current_c[j] + 1
            for k in range(prev, later):
                E[j, :] += F[k, :]
            prev = later
        F = np.zeros((m, m), dtype=int)
        prev = 0
        for j in range(m):
            later = current_c[j] + 1
            for k in range(prev, later):
                F[:, j] += E[:, k]
            prev = later
        F = F != 0
        edges[i] = np.sum(F)
    return edges


def remove_zero_row(A, n):
    # Check if any columns are zero columns
    B = np.sum(A, axis=0) == 0
    indices = [i for i in range(n) if not B[i]]
    A = A[indices][:, indices]
    return A


def AMG(fW, beta, NS):
    def fine2coarse(W, beta):
        n = W.shape[0]
        nW = W.multiply(1.0 / np.sum(W, axis=1))
        # Select coarse nodes (using ChooseCoarseGreedy_mex)
        c = ChooseCoarseGreedy_mex(nW, np.random.permutation(n), beta)

        # Compute the interp matrix
        ci = np.where(c)[0]
        P = W[:, ci]
        P = P.multiply(1.0 / np.sum(P, axis=1))

        # Make sure coarse points are directly connected to their fine counterparts
        jj, ii, pji = find(P.transpose())
        # jj, ii, pji = coo_matrix(P.transpose())
        sel = ~c[ii]
        # P = csr_matrix((np.concatenate((pji[sel], np.ones(sum(c)))), (np.concatenate((jj[sel], np.arange(sum(c)))), np.concatenate((ii[sel], ci)))), shape=(len(c), len(ci)))

        # Define mycat function
        def mycat(x, y):
            return np.concatenate((np.ravel(x), np.ravel(y)))

        # Create sparse matrix P
        rows = mycat(jj[sel], np.arange(np.sum(c)))
        cols = mycat(ii[sel], ci)
        values = mycat(pji[sel], np.ones(np.sum(c)))

        # Calculate the size of the sparse matrix P
        num_rows = np.max(rows) + 1
        num_cols = np.max(cols) + 1

        # Create a sparse matrix in CSR format
        P = csr_matrix((values, (rows, cols)), shape=(num_rows, num_cols)).transpose()

        return c, P
    n = fW.shape[0]

    P = [None] * NS
    c = [None] * NS
    W = [None] * NS

    # Make sure diagonal is zero
    W[0] = (fW - spdiags(fW.diagonal(), 0, n, n))
    fine = np.arange(n)

    for si in range(NS):
        tmp_c, P[si] = fine2coarse(W[si], beta)
        c[si] = fine[tmp_c]

        if si < NS - 1:
            W[si + 1] = csr_matrix(P[si].transpose().dot(W[si]).dot(P[si]).transpose()).transpose()

            W[si + 1] = W[si + 1] - spdiags(W[si + 1].diagonal(), [0], W[si + 1].shape[0], W[si + 1].shape[1], format='csc')
            fine = c[si]

    # Restriction matrices
    def spmtimesd(x, weights):
        return csr_matrix(x.transpose() * weights)

    R = [None] * NS
    # Apply the function to each element of P using list comprehension
    # R = [spmtimesd(x, 1. / np.sum(x, axis=1)) for x in P]
    for i, x in enumerate(P):
        x_transpose = x.transpose()
        column_sum = np.sum(x, axis=0)
        inverse_sum = 1.0 / column_sum
        result = x_transpose.multiply(inverse_sum.transpose())
        result_sparse = csr_matrix(result.transpose()).transpose()
        R[i] = result_sparse
    # R = [csr_matrix((x.transpose()).multiply(1.0 / np.sum(x, axis=0))) for x in P]
    return P, R, W, c


def ChooseCoarseGreedy_mex(nC, ord, beta):
    n = nC.shape[0]

    # allocate space for sum_jc
    sum_jc = np.zeros(n)

    # allocate space for indicator vector c
    c = np.zeros(n, dtype=bool)

    for current in ord:
        if sum_jc[current] <= beta:
            # add current to coarse and update sum_jc accordingly
            c[current] = True
            sum_jc = sum_jc + nC.getcol(current).toarray().flatten()
    return c


def DBSCAN(X, epsilon, MinPts):
    C = 0
    n = X.shape[0]
    IDX = np.zeros(n, dtype=int)

    # calculate the Euclidean distances between all points in the dataset X
    D = pairwise_distances(X, metric='euclidean')

    visited = np.zeros(n, dtype=bool)
    isnoise = np.zeros(n, dtype=bool)

    def RegionQuery(i):
        return np.where(D[i, :] <= epsilon)[0]

    def ExpandCluster(i, neighbors, C):
        IDX[i] = C

        k = 0
        while k < len(neighbors):
            j = neighbors[k]

            if not visited[j]:
                visited[j] = True
                neighbors2 = RegionQuery(j)
                if len(neighbors2) >= MinPts:
                    neighbors = np.concatenate((neighbors, neighbors2))
            if IDX[j] == 0:
                IDX[j] = C

            k += 1

    for i in range(n):
        if not visited[i]:
            visited[i] = True

            Neighbors = RegionQuery(i)
            if len(Neighbors) < MinPts:
                # X[i, :] is NOISE
                isnoise[i] = True
            else:
                C += 1
                ExpandCluster(i, Neighbors, C)

    return IDX, isnoise


def remove_lone_nodes(a):
    """Renumbers nodeIDs in order to minimize and optimize the original edges file"""
    flat_values = np.unique(np.sort(a.flatten()))
    x = {value: i for i, value in enumerate(flat_values)}
    for key, value in x.items():
        a[a == key] = value
    return a

def zero_elements(A, percentage):
    # Get the indices of elements that are 1
    one_indices = np.where(A == 1)

    # Calculate the number of elements to zero
    num_to_zero = int(len(one_indices[0]) * percentage / 100)

    # Randomly choose indices to zero
    zero_indices = np.random.choice(len(one_indices[0]), num_to_zero, replace=False)

    # Zero the elements at these indices
    A[one_indices[0][zero_indices], one_indices[1][zero_indices]] = 0

    return A

def remove_random_rows(a, percentage):
    rows_to_remove = int(len(a) * percentage / 100)
    indices_to_remove = np.random.choice(a.shape[0], size=rows_to_remove, replace=False)

    # Remove the selected rows
    a_reduced = np.delete(a, indices_to_remove, axis=0)

    return a_reduced


#TODO: need to check that the input matrix is not affected by the operations in this function
def drop_elements_wrap(arr, percentage):
    # Calculate the number of elements to drop
    num_to_drop = int(len(arr) * percentage / 100)

    # Calculate the step size for dropping elements
    step_size = len(arr) // num_to_drop

    # Create a new array by dropping elements at regular intervals
    new_arr = np.delete(arr, np.arange(0, len(arr), step_size))

    # If we haven't dropped enough elements due to rounding, drop from the beginning
    while len(new_arr) > len(arr) - num_to_drop:
        new_arr = np.delete(new_arr, 0)

    return new_arr

PyTorch

In [ ]:
import torch

def supernode_torch(A, c):
    edges = torch.zeros(len(c), dtype=torch.int)
    for i, current_c in enumerate(c):
        F = A.clone()  # Clone A to avoid modifying the original tensor
        n = F.shape[0]
        m = len(current_c)
        E = torch.zeros((m, n), dtype=torch.int)
        prev = 0
        for j in range(m):
            later = current_c[j] + 1
            for k in range(prev, later):
                E[j, :] += F[k, :]
            prev = later
        F = torch.zeros((m, m), dtype=torch.int)
        prev = 0
        for j in range(m):
            later = current_c[j] + 1
            for k in range(prev, later):
                F[:, j] += E[:, k]
            prev = later
        F = F != 0
        edges[i] = torch.sum(F)
    return edges

def remove_zero_row_torch(A):
    # Sum A across rows (dim=0) and check if any columns sum to zero
    # Note: No need to pass `n` anymore, as PyTorch can handle dynamic sizing
    col_sums = torch.sum(A, dim=0)
    row_sums = torch.sum(A, dim=1)

    # Identify non-zero columns and rows
    non_zero_cols = col_sums != 0
    non_zero_rows = row_sums != 0

    # Filter A to remove zero rows and columns
    # Note: We use boolean indexing in PyTorch to achieve this
    A_filtered = A[non_zero_rows][:, non_zero_cols]

    return A_filtered

def ChooseCoarseGreedy_mex_torch(nC, ord, beta):
    # Assuming nC is a dense tensor here. If nC is originally a sparse matrix,
    # you might need to convert it to a dense tensor if it's not too large, or handle it as a sparse tensor.
    n = nC.shape[0]

    # allocate space for sum_jc
    sum_jc = torch.zeros(n)

    # allocate space for indicator vector c
    c = torch.zeros(n, dtype=torch.bool)

    for current in ord:
        if sum_jc[current] <= beta:
            # add current to coarse and update sum_jc accordingly
            c[current] = True
            # If nC is a dense tensor, we directly use it; otherwise, convert the needed column to dense
            # For sparse nC, you'd use something like: nC[:, current].to_dense().view(-1)
            sum_jc += nC[:, current].flatten()
    return c

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# input_file_path = '/content/drive/MyDrive/Misc-Gan_Project/datasets/cora.txt'

# a = np.loadtxt(input_file_path, dtype=int)  # Read the edges from the text file
# a = remove_lone_nodes(a)
# n = np.max(a) + 1  # Find the maximum element in the entire array and add 1
# A = np.zeros((n, n), dtype=int)

# for i in range(a.shape[0]):
#     # A[a[i, 0] + 1, a[i, 1] + 1] = 1
#     A[a[i, 0], a[i, 1]] = 1
# A = remove_zero_row(A, n)

# # Convert the adjacency matrix to a sparse matrix
# A_sparse = csr_matrix(A.transpose()).transpose()
# P, R, W, c = AMG(fW=A_sparse, beta=0.2, NS=5)

# # Apply AMG clustering
# epsilon = 0.5
# MinPts = 10
# indices = DBSCAN(X=A, epsilon=epsilon, MinPts=MinPts)
# # labels = dbscan.fit_predict(pairwise_distances(A_sparse, metric='l2'))

# # Extract edges between supernodes
# edges = supernode(A, c)
# c = [[x + 1 for x in a] for a in c]
# data = {'A': A, 'P': P, 'W': W, 'R': R, 'IDX': indices, 'edges': edges, 'c': c}
# savemat('/content/drive/MyDrive/Misc-Gan_Project/data/cora.mat', data)

In [ ]:
input_file_path = '/content/drive/MyDrive/Multi-Scale_Graph_Generation_with_GU-net/data/cora.txt'

a = np.loadtxt(input_file_path, dtype=int)  # Read the edges from the text file
a = remove_lone_nodes(a)
n = np.max(a) + 1  # Find the maximum element in the entire array and add 1
A = np.zeros((n, n), dtype=int)

for i in range(a.shape[0]):
    # A[a[i, 0] + 1, a[i, 1] + 1] = 1
    A[a[i, 0], a[i, 1]] = 1

# Convert the adjacency matrix to a sparse matrix
A_sparse = csr_matrix(A.transpose()).transpose()
P, R, W, c = AMG(fW=A_sparse, beta=0.2, NS=5)

# Apply AMG clustering
epsilon = 0.5
MinPts = 10
indices = DBSCAN(X=A, epsilon=epsilon, MinPts=MinPts)
# labels = dbscan.fit_predict(pairwise_distances(A_sparse, metric='l2'))

# Extract edges between supernodes
edges = supernode(A, c)
c = [[x + 1 for x in a] for a in c]
data = {'A': A, 'P': P, 'W': W, 'R': R, 'IDX': indices, 'edges': edges, 'c': c}
savemat('/content/drive/MyDrive/Multi-Scale_Graph_Generation_with_GU-net/data/cora.mat', data)

<ipython-input-15-e4c7b792565e>:37: RuntimeWarning: divide by zero encountered in divide
  nW = W.multiply(1.0 / np.sum(W, axis=1))
<ipython-input-15-e4c7b792565e>:44: RuntimeWarning: divide by zero encountered in divide
  P = P.multiply(1.0 / np.sum(P, axis=1))


In [ ]:
!cp -r /content/data /content/drive/MyDrive/Save

In [ ]:
!rm -rf /content/drive/MyDrive/Save